# __키워드로 이미지 검색하기__ 

**키워드-이미지 검색 서비스 이해하기**
---
👉 ThanoSQL에서는 키워드를 통해 이미지를 결과로 돌려받을 수 있는 검색 기능을 제공합니다. 이미지 분류 모델 등을 이용해서 사용자가 원하는 키워드를 목표값(Target)으로 설정하고, 학습 된 모델을 사용해서 업데이트 되는 이미지를 색인한 컬럼을 추가합니다. 텍스트-이미지 검색 기능이 이미지로부터 의미를 분석하고 유사한 이미지를 제공한다면 키워드-이미지 검색은 원하는 목표값(범주)에 해당하는 이미지를 찾아줍니다. 

**본 튜토리얼에서는**   
---
👉 ThanoSQL에서 제공하는 비정형 데이터 검색 구문인 "__SEARCH__" 쿼리 구문과 기존 PostgreSQL에서 제공하는 정형 데이터 검색 구문인 "__SELECT__" 쿼리 구문을 같이 사용하여 특정 키워드를 이용해 원하는 이미지 검색을 진행합니다. 

 
**데이터 세트 설명**  
---
👉 `음식 이미지 및 영양정보 텍스트 소개` 데이터 세트는 과학기술정보통신부가 주관하고 한국지능정보사회진흥원이 지원하는 '인공지능 학습용 데이터 구축사업'으로 구축된 데이터로 한국인 다빈도 섭취 외식 메뉴와 한식메뉴 400종을 선정하여 양질의 이미지 데이터로 구성이 되어 있습니다. 842,000 장의 이미지로 구성되어 있으며, 본 튜토리얼에서는 해당 데이터 세트에서 일부(10종, 1,190장)의 사진만을 사용합니다. 

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/quick_start/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은 개인 토큰>

In [2]:
%%thanosql
COPY diet FROM "tutorial_data/diet_data/diet.csv"

Success


``` sql 
COPY [테이블 명] FROM [csv 파일]  
- 위의 쿼리는 csv 파일 데이터 세트를 ThanoSQL DB의 테이블로 만들어 줍니다.
```

## __1. 데이터 세트 확인__

키워드-이미지 검색 모델을 만들기 위해 ThanoSQL DB에 저장되어 있는 diet 테이블을 사용합니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [3]:
%%thanosql
SELECT * 
FROM diet

,image,label
0,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
1,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
2,tutorial_data/diet_data/diet/백향과/1_A220148XX_0...,백향과
3,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
4,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과
...,...,...
1185,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이
1186,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이
1187,tutorial_data/diet_data/diet/사과파이/1_A020511XX_...,사과파이
1188,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이


diet 테이블은 아래와 같은 정보를 담고 있습니다.   

-  image : 이미지 경로 
-  label : 파일 이름


## __2. 키워드 검색 모델 생성__ 

이미지 검색을 위해서는 기존 데이터 테이블을 학습하여 추후 검색의 기준을 만들어줘야 합니다. 이를 위해서 이전 단계에서 확인한 데이터 세트를 사용하여 이미지 분류 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 diet_image_classification이라는 이름의 모델을 만듭니다. 

(쿼리 실행 시 예상 소요 시간: 3 min)  


In [4]:
%%thanosql
BUILD MODEL diet_image_classification
USING ConvNeXt_Tiny
OPTIONS (
    image_col='image', 
    label_col='label', 
    epochs=1
    )
AS 
SELECT *
FROM diet 

Success


**쿼리 세부 정보**
- "__BUILD MODEL__" 쿼리 구문을 사용하여 diet_image_classification 모델을 만들고 학습시킵니다.
- "__USING__" 쿼리 구문을 통해 베이스 모델로 `ConvNeXt_Tiny`를 사용할 것을 명시합니다.
- "__OPTIONS__" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
    - "image_col" : 이미지 경로를 담은 컬럼의 이름
    - "label_col" : 목표값의 정보를 담은 컬럼의 이름
    - "epochs" : 모든 학습 데이터 세트를 학습하는 횟수 

## __3. 생성된 모델을 사용하여 키워드-이미지 검색 모델 확인__

이전 단계에서 만든 이미지 예측 모델 diet_image_classification을 사용해서 특정 이미지의 목표값을 예측해 봅니다. 아래 쿼리를 수행하고 나면, 예측 결과는 predicted 컬럼에 저장되어 반환됩니다.

In [5]:
%%thanosql
PREDICT USING diet_image_classification
AS 
SELECT *
FROM diet

,image,label,predicted
0,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
1,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,백향과
2,tutorial_data/diet_data/diet/백향과/1_A220148XX_0...,백향과,복숭아
3,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
4,tutorial_data/diet_data/diet/백향과/0_A220148XX_0...,백향과,복숭아
...,...,...,...
1185,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
1186,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,빵
1187,tutorial_data/diet_data/diet/사과파이/1_A020511XX_...,사과파이,보쌈
1188,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,복숭아


**쿼리 세부 정보**
- "__PREDICT USING__" 쿼리 구문을 통해 이전 단계에서 만든 diet_image_classification 모델을 예측에 사용합니다.

## __4. 생성된 모델을 이용한 검색__ 

이제 "PREDICT USING", "SELECT", "WHERE" 쿼리 구문을 사용하여 특정 조건의 데이터만을 검색합니다. label이 '사과파이'이고, 예측 결과 또한 '사과파이'인 데이터만을 검색하고 다음처럼 쿼리 구문을 작성할 수 있습니다.

In [6]:
%%thanosql
SELECT * 
FROM (
    PREDICT USING diet_image_classification
    AS 
    SELECT *
    FROM diet
    )
WHERE label = predicted
AND label LIKE '사과파이'
LIMIT 10

,image,label,predicted
0,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
1,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
2,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
3,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
4,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
5,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
6,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
7,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
8,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이
9,tutorial_data/diet_data/diet/사과파이/0_A020511XX_...,사과파이,사과파이


**쿼리 세부 정보**
- "__SELECT * FROM (...)__" 쿼리 구문을 통해  "__PREDICT USING__"으로 시작하는 쿼리 구문의 결과를 모두 선택합니다.
- "__WHERE__" 쿼리 구문을 통해 조건을 설정합니다. 이 조건은 "__AND__"를 통해 이어집니다.
    - "label = predicted" : label 컬럼과 predicted 컬럼의 값이 같은 데이터만 추출합니다.
    - "label = '사과파이'" : label 컬럼이 '사과파이'인 데이터만 추출합니다.